In [1]:
import sys
sys.path.append('/home/s94zalek_hpc/shape_matching')

import torch

sys.path.append('/home/s94zalek_hpc/shape_matching/pyFM_fork')
import pyFM

import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import trimesh

scene = trimesh.Scene()

In [4]:
import my_code.diffusion_training.data_loading as data_loading

test_dataset = data_loading.get_val_dataset(
    'FAUST_orig', 'train', 200
    )[1]

Loading base dataset: 100%|███████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:20<00:00,  3.99it/s]


In [5]:
from pyFM.mesh import TriMesh

data_10 = test_dataset[15]

mesh1 = TriMesh(
    data_10['first']['verts'],
    data_10['first']['faces'],
    center=True, area_normalize=True
    ).process(k=150, intrinsic=True)

mesh2 = TriMesh(
    data_10['second']['verts'],
    data_10['second']['faces'],
    center=True, area_normalize=True
    ).process(k=150, intrinsic=True)

mesh1_geo = mesh1.get_geodesic()
mesh2_geo = mesh2.get_geodesic()

In [61]:
# torch.cdist(torch.tensor(mesh1.vertices), torch.tensor(mesh1.vertices)).numpy(), mesh1_geo
# torch.cdist(data_10['first']['verts'], data_10['first']['verts']), mesh1_geo

# torch.cdist(data_10['first']['verts'], data_10['first']['verts']).max()

mesh1_geo[0][:10]

array([0.        , 0.0113947 , 0.01180185, 0.00998994, 0.01930558,
       0.02143227, 0.02016001, 0.02093965, 0.04879107, 0.05484311])

In [8]:
from pyFM.signatures.WKS_functions import mesh_WKS
from pyFM.signatures.HKS_functions import mesh_HKS
from pyFM.spectral.nn_utils import knn_query

wks_descr1 = mesh_WKS(mesh1, num_E=100, k=100)
wks_descr2 = mesh_WKS(mesh2, num_E=100, k=100)

hks_descr1 = mesh_HKS(mesh1, num_T=100, k=100)
hks_descr2 = mesh_HKS(mesh2, num_T=100, k=100)

# WKS initialisation
p2p_21_wks = knn_query(wks_descr1, wks_descr2, k=1) # (n2,) initialisation with 1-NN

# HKS initialisation
p2p_21_hks = knn_query(hks_descr1, hks_descr2, k=1) # (n2,) initialisation with 1-NN

In [20]:
# print indices where p2p_21_wks and p2p_21_hks are equal
idx_equal = np.where(p2p_21_wks == p2p_21_hks)[0]
corr_values = p2p_21_wks[idx_equal]

print('Equal indices:', idx_equal)
print('Corresponding values:', corr_values)


Equal indices: [ 249  414  768  972  974  976  977  981 1143 1472 3208 3372 3383 3384
 3387 3394 3399 3402 4456 4575 4596 4603 4642 4744 4853 5001 5011 5365
 6283 6608 6770 6799 6817]
Corresponding values: [ 414  414 1236 4988 4988 4477 4477 4477 4989 1075 3324 3440 3465 3454
 3465 3454 3403 3440 4988 1155 1076 1076 1076 5365 5010 1076 5010 5010
 4741 3324 3403 3403 3374]


In [19]:
mesh2_geo[idx_equal[i]].shape

(33, 6890)

In [23]:
distances = []

for i in range(len(idx_equal)):
    geo_dist = mesh1_geo[idx_equal[i]][corr_values[i]]
    distances.append(geo_dist)
    # print('Geodesic distance:', geo_dist)
    
print(f'Distances mean: {np.mean(distances):3f}, median: {np.median(distances):3f}')

Distances mean: 0.353800, median: 0.085387


In [39]:
closest_verts_by_descriptors = []
for i in range(len(p2p_21_hks)):
    payload = {
        'vertex1': i,
        'vertex2': p2p_21_hks[i],
        'wks_diff_min': np.linalg.norm(wks_descr1[i] - wks_descr2[p2p_21_hks[i]]),
        'hks_diff_min': np.linalg.norm(hks_descr1[i] - hks_descr2[p2p_21_hks[i]]),
    }
    payload['total_diff'] = payload['wks_diff_min'] + payload['hks_diff_min']
    
    closest_verts_by_descriptors.append(payload)
    
closest_verts_by_descriptors = sorted(closest_verts_by_descriptors, key=lambda x: x['total_diff'])

In [45]:
distances = []

for i in range(len(closest_verts_by_descriptors)):
    geo_dist = mesh2_geo[closest_verts_by_descriptors[i]['vertex1']][closest_verts_by_descriptors[i]['vertex2']]
                                                                     
    closest_verts_by_descriptors[i]['geo_dist'] = geo_dist
                                                                     
    # distances.append(geo_dist)
    # print('Geodesic distance:', geo_dist)
    
# print(f'Distances mean: {np.mean(distances):3f}, median: {np.median(distances):3f}')


In [63]:
print('mean', np.mean([x['geo_dist'] for x in closest_verts_by_descriptors[:5]]))
print('median', np.median([x['geo_dist'] for x in closest_verts_by_descriptors[:5]]))

mean 0.07159305002843927
median 0.06842663636065116
